Lab: creating simple LLMChains with memory, and Agents without memory, with the goal of: getting a working Agent with memory and a few simple tools.  PR can be in a colab notebook, or just python code that works in the REPL

In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00


In [29]:
!pip install openai

  Using cached openai-1.8.0-py3-none-any.whl (222 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
!pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.7 MB/s eta 0:00:00


# Simple Chat agent using Anyscale w/ Memory

In [36]:
from langchain_community.chat_models import ChatAnyscale
from langchain.agents import AgentExecutor, AgentType, load_tools, initialize_agent, get_all_tool_names, ZeroShotAgent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.tools import DuckDuckGoSearchRun
from langchain.chains import LLMChain
from google.colab import userdata

tool_names_used = ["llm-math", "ddg-search"]

def build_simple_agent(tool_names):
    llm = ChatAnyscale(anyscale_api_base="https://api.endpoints.anyscale.com/v1",
                    anyscale_api_key=userdata.get("ANYSCALE_API_KEY"),
                    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
                    temperature=0.7,
                    verbose=True)
    tools=load_tools(tool_names, llm=llm)
    agent=initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose=True)
    print(agent.agent.llm_chain.prompt.template)
    return agent

def chat_with_agent(agent: AgentExecutor, user_input: str, handle_parsing_errors=True):
    output=agent.invoke({'input': user_input})
    return output




Interacting with my chat agent

In [30]:
chat_with_agent(build_simple_agent(["llm-math", "ddg-search"]),{"input":"What is weather like today in Caldwell Idaho?"})

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, duckduckgo_search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


> Entering new AgentExecutor chain...
 I need to find out the current weather in Caldwell, Idaho. I will use the duckduckgo_search tool for this.

Action: duckduckgo_search
Action Input: 

{'input': {'input': 'What is weather like today in Caldwell Idaho?'},
 'output': 'The weather in Caldwell, Idaho is slightly cloudy with a wind of 16 mph from the southeast, humidity of 71%, and a 5% probability of precipitation. The daytime high is a cold 39.2°F, and the night low is a freezing 30.2°F.'}

Testing the Agent's ability to do math

In [31]:
chat_with_agent(build_simple_agent(["llm-math", "ddg-search"]),{"input":"what is the derivative of y=e^x"})

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, duckduckgo_search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


> Entering new AgentExecutor chain...
 The derivative of a function in the form of y = e^x is a fundamental concept in calculus. I can answer this question without external resources.

Fi

{'input': {'input': 'what is the derivative of y=e^x'},
 'output': "The derivative of the function y = e^x is y' = e^x."}

# Creating simple LLMChains with memory

Let’s walk through the docs on usage of memory with simple LLM and Chat “chains” (simpler than agents, have no tools or loops, just PromptTemplates)


In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = ChatAnyscale(anyscale_api_base="https://api.endpoints.anyscale.com/v1",
                    anyscale_api_key=userdata.get("ANYSCALE_API_KEY"),
                    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
                    temperature=0.7,
                    verbose=True)
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [23]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi there! what formation is the best in a soccer match?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi there! how formation is the best in a soccer match?
AI:  Hello! It's great to chat with you about soccer. When it comes to formation in a soccer match, the "best" one can vary greatly depending on the team's strengths, weaknesses, the opposing team's strategies, and other factors. However, some commonly used and successful formations include the 4-4-2 (four defenders, four midfielders, and two forwards), the 4-3-3 (four defenders, three midfielders, and three forwards), and the 4-2-3-1 (four defenders, two defensive midfielders, three attacking midfielders, and one forward).

The 4-4-2 formation is balanced and flexible, allowing for both defensive stability and offensive pressure. The 4-3-3 formation is often used by teams that want to dominate possession and control the game, while the 4-2-3-1 formation is great for teams that want t

{'question': 'hi there! what formation is the best in a soccer match?',
 'chat_history': 'Human: hi there! how formation is the best in a soccer match?\nAI:  Hello! It\'s great to chat with you about soccer. When it comes to formation in a soccer match, the "best" one can vary greatly depending on the team\'s strengths, weaknesses, the opposing team\'s strategies, and other factors. However, some commonly used and successful formations include the 4-4-2 (four defenders, four midfielders, and two forwards), the 4-3-3 (four defenders, three midfielders, and three forwards), and the 4-2-3-1 (four defenders, two defensive midfielders, three attacking midfielders, and one forward).\n\nThe 4-4-2 formation is balanced and flexible, allowing for both defensive stability and offensive pressure. The 4-3-3 formation is often used by teams that want to dominate possession and control the game, while the 4-2-3-1 formation is great for teams that want to maintain a solid defense while also having a 

In [24]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation.invoke({"question": "what was discussed in our previous conversation?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi there! how formation is the best in a soccer match?
AI:  Hello! It's great to chat with you about soccer. When it comes to formation in a soccer match, the "best" one can vary greatly depending on the team's strengths, weaknesses, the opposing team's strategies, and other factors. However, some commonly used and successful formations include the 4-4-2 (four defenders, four midfielders, and two forwards), the 4-3-3 (four defenders, three midfielders, and three forwards), and the 4-2-3-1 (four defenders, two defensive midfielders, three attacking midfielders, and one forward).

The 4-4-2 formation is balanced and flexible, allowing for both defensive stability and offensive pressure. The 4-3-3 formation is often used by teams that want to dominate possession and control the game, while the 4-2-3-1 formation is great for teams that want t

{'question': 'what was discussed in our previous conversation?',
 'chat_history': 'Human: hi there! how formation is the best in a soccer match?\nAI:  Hello! It\'s great to chat with you about soccer. When it comes to formation in a soccer match, the "best" one can vary greatly depending on the team\'s strengths, weaknesses, the opposing team\'s strategies, and other factors. However, some commonly used and successful formations include the 4-4-2 (four defenders, four midfielders, and two forwards), the 4-3-3 (four defenders, three midfielders, and three forwards), and the 4-2-3-1 (four defenders, two defensive midfielders, three attacking midfielders, and one forward).\n\nThe 4-4-2 formation is balanced and flexible, allowing for both defensive stability and offensive pressure. The 4-3-3 formation is often used by teams that want to dominate possession and control the game, while the 4-2-3-1 formation is great for teams that want to maintain a solid defense while also having a lot of 

In [25]:
# I was curious to know what the LLM Chain know about seinfeld show
conversation({"question": "Give a summary of season 3 in the Seinfeld show"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi there! how formation is the best in a soccer match?
AI:  Hello! It's great to chat with you about soccer. When it comes to formation in a soccer match, the "best" one can vary greatly depending on the team's strengths, weaknesses, the opposing team's strategies, and other factors. However, some commonly used and successful formations include the 4-4-2 (four defenders, four midfielders, and two forwards), the 4-3-3 (four defenders, three midfielders, and three forwards), and the 4-2-3-1 (four defenders, two defensive midfielders, three attacking midfielders, and one forward).

The 4-4-2 formation is balanced and flexible, allowing for both defensive stability and offensive pressure. The 4-3-3 formation is often used by teams that want to dominate possession and control the game, while the 4-2-3-1 formation is great for teams that want t

{'question': 'Give a summary of season 3 in the Seinfeld show',
 'chat_history': 'Human: hi there! how formation is the best in a soccer match?\nAI:  Hello! It\'s great to chat with you about soccer. When it comes to formation in a soccer match, the "best" one can vary greatly depending on the team\'s strengths, weaknesses, the opposing team\'s strategies, and other factors. However, some commonly used and successful formations include the 4-4-2 (four defenders, four midfielders, and two forwards), the 4-3-3 (four defenders, three midfielders, and three forwards), and the 4-2-3-1 (four defenders, two defensive midfielders, three attacking midfielders, and one forward).\n\nThe 4-4-2 formation is balanced and flexible, allowing for both defensive stability and offensive pressure. The 4-3-3 formation is often used by teams that want to dominate possession and control the game, while the 4-2-3-1 formation is great for teams that want to maintain a solid defense while also having a lot of o

In [26]:
# Relating the two previous chats about soccer and seinfeld to create an episode about soccer

conversation.invoke({"question":"Using the previous conversations create a show which is based on the show I asked about the sports which I asked earlier"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: hi there! how formation is the best in a soccer match?
AI:  Hello! It's great to chat with you about soccer. When it comes to formation in a soccer match, the "best" one can vary greatly depending on the team's strengths, weaknesses, the opposing team's strategies, and other factors. However, some commonly used and successful formations include the 4-4-2 (four defenders, four midfielders, and two forwards), the 4-3-3 (four defenders, three midfielders, and three forwards), and the 4-2-3-1 (four defenders, two defensive midfielders, three attacking midfielders, and one forward).

The 4-4-2 formation is balanced and flexible, allowing for both defensive stability and offensive pressure. The 4-3-3 formation is often used by teams that want to dominate possession and control the game, while the 4-2-3-1 formation is great for teams that want t

{'question': 'Using the previous conversations create a show which is based on the show I asked about the sports which I asked earlier',
 'chat_history': 'Human: hi there! how formation is the best in a soccer match?\nAI:  Hello! It\'s great to chat with you about soccer. When it comes to formation in a soccer match, the "best" one can vary greatly depending on the team\'s strengths, weaknesses, the opposing team\'s strategies, and other factors. However, some commonly used and successful formations include the 4-4-2 (four defenders, four midfielders, and two forwards), the 4-3-3 (four defenders, three midfielders, and three forwards), and the 4-2-3-1 (four defenders, two defensive midfielders, three attacking midfielders, and one forward).\n\nThe 4-4-2 formation is balanced and flexible, allowing for both defensive stability and offensive pressure. The 4-3-3 formation is often used by teams that want to dominate possession and control the game, while the 4-2-3-1 formation is great for

# Simple Chat Agent with Memory



In [51]:

def build_agent_w_memory():
    search = DuckDuckGoSearchRun()
    tools = [
        Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]
    llm = ChatAnyscale(anyscale_api_base="https://api.endpoints.anyscale.com/v1",
                    anyscale_api_key=userdata.get("ANYSCALE_API_KEY"),
                    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
                    temperature=0.7,
                    verbose=True)
    prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
    suffix = """Begin!"

    {chat_history}
    Question: {input}
    {agent_scratchpad}
    """

    prompt = ZeroShotAgent.create_prompt(
        tools,
        prefix=prefix,
        suffix=suffix,
        input_variables=["input", "chat_history", "agent_scratchpad"],
    )
    memory = ConversationBufferMemory(memory_key="chat_history")

    llm_chain = LLMChain(llm=llm, prompt=prompt)
    agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

    agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
    )

    return agent_chain

agent_w_memory = build_agent_w_memory()






# Testing out agent with memory

In [52]:
agent_w_memory.invoke("How many people live in Ghana?")



> Entering new AgentExecutor chain...
 Thought: I don't have real-time data, but I can provide an estimate based on the most recent data available.
Action: Search
Action Input: Current population of Ghana
Observation
Observation: Demographic and Health Survey 2022 Abbreviation or Acronym DHS/ GDHS 2022 Country/Economy Study type Demographic and Health Survey [hh/dhs] Series Information The 2022 Ghana Demographic and Health Survey (2022 GDHS) is the seventh of its kind and follows those implemented in 1988, 1993, 1998, 2003, 2008, and 2014. Population Age structure Birth and fertility rate Mortality Households and ownership Poverty Editor's Picks Current statistics on this topic Population Share of population in Ghana 2021,... The total population in Ghana was forecast to continuously increase between 2023 and 2028 by in total 4.4 million people (+13.37 percent). After the tenth consecutive increasing year, the... Home Population Africa Ghana Ghana Population Population Time Moon Geog

{'input': 'How many people live in Ghana?',
 'chat_history': '',
 'output': 'As of my last training data in 2021, the population of Ghana was approximately 34.47 million. However, I found estimates from reliable sources that the population in Ghana continues to increase. As of January 15, 2024, the population is forecasted to be around 34.47 million + 4.4 million (13.37% increase) which is approximately 38.87 million. Please note that this is an approximation and the actual number might vary slightly.'}

In [54]:
agent_w_memory("what is their national anthem?")



> Entering new AgentExecutor chain...
 Thought: I don't have this information readily available, so I will need to search for the national anthem of Ghana.
Action: Search
Action Input: Ghana national anthem

Observation
Observation: 1. Verse 1: "God bless our homeland Ghana, and make our nation great and strong." This line reflects the deep-rooted faith of Ghanaians, seeking divine guidance and blessings for the growth and prosperity of their beloved nation. 2. Verse 2: "Help us to resist oppressors' rule with all our will and might forevermore." JUL 18, 2023 LISTEN In today's age of viral videos and social media, public figures are constantly under scrutiny, with their actions and behavior subject to intense observation. . It was on Friday, June 9, 2023, during the Green Ghana Day event at the University of Ghana, attended by the president and the Minister of the Greater Accra Region, Henry Quartey, plus other... 13.06.2023 LISTEN The Chief of Mempeasem near Legon in Accra, Nii Torg

{'input': 'what is their national anthem?',
 'chat_history': 'Human: How many people live in Ghana?\nAI: As of my last training data in 2021, the population of Ghana was approximately 34.47 million. However, I found estimates from reliable sources that the population in Ghana continues to increase. As of January 15, 2024, the population is forecasted to be around 34.47 million + 4.4 million (13.37% increase) which is approximately 38.87 million. Please note that this is an approximation and the actual number might vary slightly.',
 'output': 'The national anthem of Ghana is called "God Bless Our Homeland Ghana." The lyrics include lines such as "God bless our homeland Ghana, and make our nation great and strong," "Fill our hearts with true humility," "Make us cherish fearless honesty," and "Help us to resist oppressors\' rule with all our will and might forevermore." The anthem was written by Emmanuel Pappoe-Thompson with music by Philip Gbeho.'}